In [ ]:
# Запуск/Остановка SparkSession
try:
    spark.stop()
    del spark
except:
    %run configuration.py

In [ ]:
# Сборка витрины по расчётным счетам корпоративных клиентов
# Обработка данных из разных источников и сохранение результата
# в таблицу Hive (на кластер)
try:
    spark.sql('drop table stor.rko_showcase')
except:
    pass

# Префикс к таблицам
prx_epk: str = 'arn_model_party_organization_'

# БД
schema: dict = {
    'rko': 'with_db_collection_sch_rko_platform_rko',
    'epk': 'with_db_collection_sch_epk_platform_uckpb',
    'sbbol': 'with_db_collection_vnp_sbbol_birdabi'
}

# Таблицы
table: dict = {
    'currency': 'platform_gdp_model_jpa_currency',
    'rstatus': 'platform_gdp_model_jpa_rstatus',
    'rtype': 'platform_gdp_model_jpa_rtype',
    'register': 'platform_gdp_model_jpa_register',
    'mregister': 'platform_gdp_model_jpa_mregister',
    'sourcesystem': 'platform_gdp_model_jpa_sourcesystem',
    'ptarget': 'platform_gdp_model_jpa_ptarget',
    'product': 'platform_gdp_model_jpa_product',
    'ptype': 'platform_gdp_model_jpa_ptype',
    'mproduct': 'platform_gdp_model_jpa_mproduct',
    'segment': 'platform_gdp_model_jpa_segment',
    'priority': 'platform_gdp_model_jpa_priority',
    'country': 'platform_gdp_model_jpa_country',
    'org': 'platform_gdp_model_jpa_org',
    'orgn': 'platform_gdp_model_jpa_orgn',
    'crm': 'platform_gdp_model_jpa_crm',
    'ogrn': 'platform_gdp_model_jpa_ogrn',
    'inn': 'platform_gdp_model_jpa_inn'
}

# Валюта счёта
currency: DataFrame = (
    spark.table(f"{schema['rko']}.{table['country']}")
    .select(
        F.col('key').alias('currency_key'),
        F.col('name').alias('currency_name')
    )
)

# Статус счёта
register_status: DataFrame = (
    spark.table(f"{schema['rko']}.{table['rstatus']}")
    .select(
        F.col('key').alias('register_status_key'),
        F.col('name').alias('register_status_name')
    )
)

# Тип счёта
register_type: DataFrame = (
    spark.table(f"{schema['rko']}.{table['rtype']}")
    .select(
        F.col('key').alias('register_type_key'),
        F.col('name').alias('register_type_name')
    )
)

# Счета
register: DataFrame = (
    spark.table(f"{schema['rko']}.{['register']}")
    .filter(
        (F.to_date(F.col('begindate'))
        .between('2024-01-01 00:00:00:000000', '2024-12-31 23:59:60:999999'))
        & (F.col('currency_short_foreign_key').like('RUR'))
        & (F.col('status_short_foreign_key').isin('Working', 'Closed'))
        & (F.col('number').substr(1, 3).isin('401', '402', '404', '405',
                                             '406', '407', '408'))
        & (F.col('number').substr(1, 5).isin('40109', '40111', '40803',
                                             '40810', '40813', '40817',
                                             '40820', '40824') == False)
    )
    .select(
        F.col('ownerepkid_entityd').cast(T.StringType()).alias('epk_id'),
        F.col('begindate').cast(T.DateType()),
        F.col('enddate').cast(T.DateType()),
        F.col('lastchangedate').cast(T.DateType()),
        F.col('number').cast(T.StringType()),
        F.col('currency_short_foreign_key').alias('currency_key'),
        F.col('status_short_foreign_key').alias('register_status_key'),
        F.col('registertype_short_foreign_key').alias('register_type_key'),
        F.col('product_short_foreign_key').alias('product_key'),
        F.col('key').alias('register_key')
    )
    .withColumn(
        'diff',
        F.datediff(
            end=(F.col('enddate')),
            start=(F.col('startdate'))
        )
    )
    .filter((F.col('diff') >= F.lit(7)) | (F.col('enddate').isNull()))
    .drop('diff')
    .join(F.broadcast(currency), on='currency_key',
          how='left_outer')
    .drop('currency_key')
    .join(F.broadcast(register_status), on='register_status_key',
          how='left_outer')
    .drop('register_status_key')
    .join(F.broadcast(register_type), on='register_type_key',
        how='left_outer')
    .drop('register_type_key')
)

# Справочник систем источников
sourcesystem: DataFrame = (
    spark.table(f"{schema['rko']}.{table['sourcesystem']}")
    .select(
        F.col('objectid').alias('sourcesystem_key'),
        F.col('name').alias('sourcesystem_name')
    )
)

# Таблица миграции счетов
mregister: DataFrame = (
    spark.table(f"{schema['rko']}.{table['mregister']}")
    .select(
        F.col('eksid').cast(T.StringType()).alias('eks_id'),
        F.col('pprbid').cast(T.StringType()).alias('pprb_id'),
        F.col('register_entityid').alias('register_key'),
        F.col('sourcesystem_short_foreign_key').alias('sourcesystem_key'),
        F.col('migratetm').alias('migration_date')
    )
    .join(register, on='register_key', how='inner')
    .drop('register_key')
    .join(F.broadcast(sourcesystem), on='sourcesystem_key', how='inner')
    .drop('sourcesystem_key')
)

# Целевое назначение договора РКО
producttarget: DataFrame = (
    spark.table(f"{schema['rko']}.{table['ptarget']}")
    .select(
        F.col('description').alias('prod_target'),
        F.col('key').alias('key_target')
    )
)

# Вид договора РКО
producttype: DataFrame = (
    spark.table(f"{schema['rko']}.{table['ptype']}")
    .select(
        F.col('name').alias('ptype_name'),
        F.col('key').alias('ptype_key')
    )
)

# Договор расчётно-кассового обслуживания ЮЛ/ИП
product: DataFrame = (
    spark.table(f"{schema['rko']}.{table['product']}")
    .filter(
        F.col('target_short_foreign_key')
        .isin(
            'CorporateCreditAccount', 'IndividualElectionFunds',
            'ElectionCommissionAccount', 'MetalPreciousAccount',
            'NotaryDeposit', 'ElectoralAssociationAccount', 
            'LawEnforcementDepositAccount', 'SpecialBrokerageAccount',
            'SpecialDepositaryAccount', 'PrivatePensionReservesAccount',
            'PaymentIdentification', 'NominalGuardian48FZ',
            'PrivatePensionSavingsAccount', 'ClearingBankAccount',
            'CurrentBailiffsServiceDepositAccount', 'IndAccount',
            'FundsTemporarityDisposal', 'UFKvalyataAccount'
        ) == False
    )
    .select(
        F.col('key').alias('product_key'),
        F.col('target_short_foreign_key').alias('key_target'),
        F.col('num').cast(T.StringType()).alias('num_dog'),
        F.col('producttype_short_foreign_key').alias('ptype_key')
    )
    .join(F.broadcast(producttarget), on='key_target', how='inner')
    .drop('key_target')
    .join(F.broadcast(producttype), on='ptype_key', how='left_outer')
    .drop('ptype_key')
    .join(mregister, on='product_key', how='inner')
)

# Справочник подразделений банка
branch: DataFrame = (
    spark.table(f"{schema['sbbol']}.branch")
    .select(
        F.col('fullname').alias('division'),
        F.col('sclir').alias('divisionekscode')
    )
)

# Дополнительные атрибуты для РКО из устаревших систем
mproduct: DataFrame = (
    spark.table(f"{schema['rko']}.{table['mproduct']}")
    .select(
        'divisionekscode',
        F.col('product_entityid').cast(T.StringType()).alias('product_key')
    )
    .join(product, on='product_key', how='right_outer')
    .drop('product_key')
    .join(branch, on='divisionekscode', how='inner')
    .drop('divisionekscode')
)

# Справочник сегментов
dictionary_segment: DataFrame = (
    spark.table(f"{schema['epk']}.{table['segment']}")
    .select(
        F.col('name').alias('segment'),
        F.col('key').alias('key_segment')
    )
)

# Справочник приоритетов
dictionary_priority: DataFrame = (
    spark.table(f"{schema['epk']}.{table['priority']}")
    .select(
        F.col('name').alias('priority'),
        F.col('key').alias('key_priority')
    )
)

# Справочник стран
dictionary_country: DataFrame = (
    spark.table(f"{schema['epk']}.{table['country']}")
    .select(
        F.col('name').alias('country_name'),
        F.col('key').alias('key_country')
    )
)

# Организации
organization: DataFrame = (
    spark.table(f"{schema['epk']}.{table['org']}")
    .select(
        F.col('id').cast(T.StringType()).alias('epk_id'),
        F.col('key').alias('key_name'),
        F.col('countryresident_short_foreign_key').alias('key_country')
    )
    .join(mproduct, on='epk_id', how='inner')
)

# Наименование организаций
organizationname: DataFrame = (
    spark.table(f"{schema['epk']}.{prx_epk}{table['orgn']}")
    .filter(
        F.col('nametype_short_foreign_key').like('NameType_3')
    )
    .select(
        'key',
        F.col('name').alias('name_org')
    )
    .withColumn('key_name', F.split('key', '\.')[0])
    .join(organization, on='key_name', how='inner')
    .drop('key')
)

# CRM-атрибуты
crmattributes: DataFrame = (
    spark.table(f"{schema['epk']}.{prx_epk}{table['crm']}")
    .select(
        F.col('id').cast(T.StringType()).alias('epk_id'),
        F.col('segment_short_foreign_key').alias('key_segment'),
        F.col('priority_short_foreign_key').alias('key_priority')
    )
    .join(organizationname, on='epk_id', how='right_outer')
)

# ОГРН клиента
win_ogrn: any = (
    Window.partitionBy('key_name', 'ogrn').orderBy(F.desc('startdate'))
)
ogrn: DataFrame = (
    spark.table(f"{schema['epk']}.{prx_epk}{table['ogrn']}")
    .select(
        F.col('documentnumber').cast(T.StringType()).alias('ogrn'),
        'key',
        'startdate'
    )
    .witnColumn('key_name', F.split('key', '\.')[0])
    .withColumn('rn', F.row_number().over(win_ogrn))
    .filter(F.col('rn') == F.lit(1)).drop('rn')
    .join(crmattributes, on='key_name', how='right_outer')
    .drop('key', 'startdate')
)

# ИНН клиента
win_inn: any = (
    Window.partitionBy('key_name', 'inn').orderBy(F.desc('startdate'))
)
inn: DataFrame = (
    spark.table(f"{schema['epk']}.{table['inn']}")
    .select(
        F.col('documentnumber').cast(T.StringType()).alias('inn'),
        'key',
        'startdate'
    )
    .witnColumn('key_name', F.split('key', '\.')[0])
    .withColumn('rn', F.row_number().over(win_inn))
    .filter(F.col('rn') == F.lit(1)).drop('rn')
    .join(ogrn, on='key_name', how='right_outer')
    .drop('key', 'startdate', 'key_name')
)

# Сборка финального DataFrame
assembling: DataFrame = (
    inn
    .withColumn('migration', F.concat_ws(' > ',
                                         'system_name',
                                         'migration_date')
    )
    .join(F.broadcast(dictionary_segment), on='key_segment',
          how='left_outer')
    .drop('key_segment')
    .join(F.broadcast(dictionary_priority), on='key_priority',
          how='left_outer')
    .drop('key_priority')
    .join(F.broadcast(dictionary_country), on='key_country',
          how='left_outer')
    .drop('key_country')
)

# Задаём порядок/структуру фрейма данных
column_dict: dict = {
    'eks_id': 'Уникальный идентификатор ЕКС',
    'pprb_id': 'Уникальный идентификатор ППРБ',
    'epk_id': 'Уникальный идентификатор ЕПК',
    'name_org': 'Наименование клиента', 'inn': 'ИНН', 'ogrn': 'ОГРН',
    'country_name': 'Страна клиента', 'segment': 'Сегмент',
    'priority': 'Приоритет', 'number': 'Номер счёта',
    'num_dog': 'Номер договора', 'begindate': 'Дата открытия счёта',
    'lastchangedate': 'Дата последнего изменения',
    'enddate': 'Дата закрытия счёта',
    'prod_target': 'Целевое назначение договора',
    'ptype_name': 'Наименование вида договора',
    'currency_name': 'Валюта счёта',
    'register_status_name': 'Статус счёта', 'register_type_name': 'Тип счёта',
    'division': 'Подразделение',
    'migration': 'Система миграции и дата миграции'
}

# Вносим корректировки и сохраняем результат в командное пространство
createSchema(
    column_dict, assembling.select(*column_dict).orderBy('begindate')
).repartition(1).write.mode('overwrite').saveAsTable('stor.rko_showcase')

In [ ]:
# Обрабатываем sql-файл и используем его в качестве запроса.
# Затем выгружаем результат в excel-файл.
code_list: list = list(open('sql_request.sql'))
sql_script: str = '\n '.join(code_list)
spark.sql(sql_script).toPandas().to_excel('crm.xlsx', index=False)

In [ ]:
# Получение выписки со всеми операциями по необходимым клиентам.
# Так как таблицы client и register будут использоваться много раз
# без изменений, логичнее их кэшировать.
# В данном скрипте используется метод оптимизации запроса через python-цикл.
# Поскольку таблицы в скрипте наполнены огромным кол-вом данных,
# чтобы избежать нагрузки "бутылочного горлышка" (bottleneck) и,
# как следствие, увеличение плана запроса и падения производительности,
# необходимо применить следующий вариант исполнения:

# Получаем фрейм с необходимым списком клиентов
client_list: DataFrame = pd.read_excel('org.xlsx', dtype=str)
org_df: DataFrame = spark.createDataFrame(client_list)

# БД
database: dict = {
    'stmnt': 'prx_collections_platform_data_api'
}

# Таблицы
tables: dict = {
    'client': 'pprb_platform_go_dataspace_client',
    'register': 'pprb_platform_go_dataspace_register',
    'turn': 'pprb_platform_go_dataspace_turn',
    'docdata': 'pprb_platform_go_dataspace_docdata'
}

# Клиенты
client: DataFrame = (
    spark.table(f"{database['stmnt']}.{tables['client']}")
    .select(
        F.col('objectid').cast(T.StringType()).alias('client'),
        F.col('ccepk').cast(T.StringType()).alias('epk_id'),
        F.col('ccinn').cast(T.StringType()).alias('inn'),
        F.col('cckpp').cast(T.StringType()),
        'ccname'
    )
    .join(F.broadcast(org_df),
          on=((F.col('epk_id') == org_df.epk_id)
              & (F.col('inn') == org_df.inn)),
          how='inner'
    )
    .dropDuplicates(['epk_id', 'inn'])
    .cache() # Для более гибкой настройки лучше использовать persist().
             # Сохранение в память:
             #     df.persist(StorageLevel.MEMORY_ONLY) <- аналог cache()
             # Сохранение на диск:
             #     df.persist(StorageLevel.DISK_ONLY)
             # Сохранение в память и на диск:
             #     df.persist(StorageLevel.MEMORY_AND_DISK)
)

# Журнал записей
register: DataFrame = (
    spark.table(f"{database['stmnt']}.{tables['register']}")
    .select(
        F.col('objectid').cast(T.StringType()).alias('register'),
        F.col('client').cast(T.StringType()),
        F.col('ccaccnum').cast(T.StringType())
    )
    .join(F.broadcast(client), on='client', how='inner')
    .drop('client')
    .cache()
)

# Словарь названий столбцов БД/Excel
column_dict: dict = {
    'ccpurpose': 'Назначение платежа',
    'ccdtacc': 'Счёт плательщика',
    'ccdtbic': 'БИК плательщика',
    'ccdtinn': 'ИНН плательщика',
    'ccdtkpp': 'КПП плательщика',
    'ccdtname': 'Наименование плательщика',
    'ccdtnamebank': 'Банк плательщика',
    'ccktacc': 'Счёт получателя',
    'ccktbic': 'БИК получателя',
    'ccktinn': 'ИНН получателя',
    'ccktkpp': 'КПП получателя',
    'ccktname': 'Наименование получателя',
    'ccktnamebank': 'Банк получателя',
    'ccstartsum': 'Входящий остаток на момент проводки',
    'ccstartsumnat': 'Входящий остаток в нац. валюте',
    'ccsum': 'Сумма проводки',
    'ccsumnat': 'Сумма в нац. валюте',
    'ccdt': 'Признак дебетования',
    'ccdate': 'Дата проводки',
    'ccaccnum': 'Номер счёта клиента',
    'inn': 'ИНН клиента',
    'ccname': 'Наименование клиента',
    'epk_id': 'ЕПК клиента',
    'cckpp': 'КПП клиента'
}

# Даты для разбивки плана запроса на 12 частей (1 год)
list_of_dates: list = [
    '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
    '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12'
]

# Самые большие таблицы прогоняем через цикл по датам разбивки.
# В качестве полей для фильтрации используем поля партиций: ctl_ccdate_part
#                                                           ctl_ccdatedoc_part
with pd.ExcelWriter('bank_statements.xlsx', engine='xlsxwriter') as writer:
    for year_month in list_of_dates:
        # Для детализации выполнения запроса делаем засечку по времени
        create_date: any = dt.datetime.now()

        # Проводка
        turn: DataFrame = (
            spark.table(f"{database['stmnt']}.{tables['turn']}")
            .filter(
                F.col('ctl_ccdate_part').like(f"{year_month}%")
            )
            .select(
                F.col('objectid').cast(T.StringType()).alias('turn'),
                'ccstartsum',
                'ccstartsumnat',
                'ccsum',
                'ccsumnat',
                'ccdt',
                F.col('objectid').cast(T.StringType()).alias('register'),
                'ccdate'
            )
            .join(F.broadcast(register), on='register', how='inner')
            .drop('register')
        )

        # Документы
        docdate: DataFrame = (
            spark.table(f"{database['stmnt']}.{tables['docdata']}")
            .filter(
                F.col('ctl_ccdatedoc_part').like(f"{year_month}%")
            )
            .select(
                'ccpurpose',
                F.col('turn').cast(T.StringType()),
                F.col('ccdtacc').cast(T.StringType()),
                F.col('ccdtbic').cast(T.StringType()),
                F.col('ccdtinn').cast(T.StringType()),
                F.col('ccdtkpp').cast(T.StringType()),
                'ccdtname',
                'ccdtnamebank',
                F.col('ccktacc').cast(T.StringType()),
                F.col('ccktbic').cast(T.StringType()),
                F.col('ccktinn').cast(T.StringType()),
                F.col('ccktkpp').cast(T.StringType()),
                'ccktname',
                'ccktnamebank'
            )
            .join(turn, on='turn', how='inner')
            .drop('turn')
        ).toPandas().rename(columns=column_dict)
        getExcel(writer, year_month, docdate)
        # Ещё одна засечка в конце, вычисляем время выполнения подзапроса
        # и выводим на экран с описанием даты и времени.
        end_date: any = dt.datetime.now()
        total_time: any = end_date - create_date
        print(f"Выписка за дату {year_month} готова через {total_time}")

# Очищаем кэш
dataframes: list = [client, register]
for df in dataframes:
    df.unpersist()